In [4]:
from whatstk import WhatsAppChat
import pandas as pd
import matplotlib.pyplot as plt

# Caminho para o arquivo de chat
filepath = r"C:\Users\breno\whatsappanalyzer\_chat 8.txt"

# Especificando o formato do cabeçalho
hformat = '[%d/%m/%y, %H:%M:%S] %name:'

# Função para carregar e filtrar o DataFrame por período de tempo
def carregar_e_filtrar_chat(filepath, hformat, inicio=None, fim=None):
    chat = WhatsAppChat.from_source(filepath=filepath, hformat=hformat)
    df = chat.df
    
    if inicio:
        df = df[df['date'] >= pd.to_datetime(inicio)]
    if fim:
        df = df[df['date'] <= pd.to_datetime(fim)]
    
    return df


In [ ]:
# Define o período de tempo desejado
inicio = '2024-01-01'
fim = '2025-01-01'

try:
    # Carregar e filtrar o chat
    df = carregar_e_filtrar_chat(filepath, hformat, inicio, fim)
    
    # Agrupando por 'username', contando as mensagens e ordenando por ordem decrescente de contagem
    sorted_df = df.groupby('username')['message'].count().sort_values(ascending=False)
    
    print(f"\nContagem de mensagens enviada por cada membro no período de {inicio} a {fim}:")
    print(sorted_df)
    
except KeyError as e:
    print("Erro de chave:", e)
except RuntimeError as e:
    print("Erro de execução:", e)
except Exception as e:
    print("Outro erro ocorreu:", e)

In [11]:
# Função para exportar o DataFrame filtrado para um arquivo CSV
def exportar_para_csv(df, caminho_csv):
    df.to_csv(caminho_csv, index=False)

try:
    # Carregar e filtrar o chat
    df = carregar_e_filtrar_chat(filepath, hformat, inicio, fim)
    # Exportar o DataFrame filtrado para um arquivo CSV
    exportar_para_csv(df, 'chat_filtrado.csv')
    print("Arquivo CSV exportado com sucesso.")
except Exception as e:
    print("An error occurred:", e)

Arquivo CSV exportado com sucesso.


In [ ]:
# Cria uma análise de quem mais falou, quebrado por mês.

# Certifique-se de que a coluna de data está no formato datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

# Criar uma nova coluna para o ano e o mês
df['year_month'] = df['date'].dt.to_period('M')
# Agrupar pelo nome do usuário e pelo mês e contar as mensagens
message_counts_by_month = df.groupby(['username', 'year_month']).size().reset_index(name='message_count')

# Ordenar o resultado por nome do usuário e data
message_counts_by_month = message_counts_by_month.sort_values(by=['username', 'year_month'])

print(message_counts_by_month)

In [17]:
#download de quem mais fala, por mês.
caminho_arquivo = '/Users/brenojaruzo/Downloads/message_counts_by_month2.csv'
message_counts_by_month.to_csv(caminho_arquivo, index=False)

In [ ]:
from whatstk import FigureBuilder
from plotly.offline import plot
fb = FigureBuilder(df=df)
fig = fb.user_interventions_count_linechart(cumulative=True, msg_length=True, title='Count of sent characters (cumulative)')
plot(fig)

In [ ]:
# Agrupar as mensagens por usuário e calcular a contagem total de caracteres em cada grupo
contagem_caracteres_por_usuario = df.groupby('username')['message'].apply(lambda x: x.str.len().sum()).reset_index()

# Renomear as colunas
contagem_caracteres_por_usuario.columns = ['Usuário', 'Contagem de Caracteres']

# Ordenar a tabela pela contagem de caracteres em ordem decrescente
contagem_caracteres_por_usuario = contagem_caracteres_por_usuario.sort_values(by='Contagem de Caracteres', ascending=False)

# Resetar o índice
contagem_caracteres_por_usuario = contagem_caracteres_por_usuario.reset_index(drop=True)

# Exibir a tabela
print(contagem_caracteres_por_usuario)

In [ ]:
#boxplot com o tamanho médio das mensagens enviadas

fig = fb.user_msg_length_boxplot()
plot(fig)

In [ ]:
#interação entre os usuários, enviando respostas após o outro

fig = fb.user_message_responses_heatmap()
plot(fig)

In [ ]:
# Extrair a hora de cada mensagem para gerar o gráfico de atividade por hora
df['hour'] = df['date'].dt.hour

# Contar o número de mensagens por hora
activity_by_hour = df['hour'].value_counts().sort_index()

# Plotar o gráfico de atividade por hora
plt.figure(figsize=(10, 6))
activity_by_hour.plot(kind='bar', color='skyblue')
plt.title('Atividade no Grupo por Hora do Dia')
plt.xlabel('Hora do Dia')
plt.ylabel('Número de Mensagens')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Análise de atividade por hora e por usuário
activity_by_user_hour = df.groupby(['username', 'hour']).size().unstack(fill_value=0)

# Plotar o gráfico de atividade por hora para cada usuário
plt.figure(figsize=(14, 8))
activity_by_user_hour.T.plot(kind='bar', stacked=True, figsize=(14, 8), colormap='viridis')
plt.title('Atividade por Hora do Dia para Cada Usuário')
plt.xlabel('Hora do Dia')
plt.ylabel('Número de Mensagens')
plt.xticks(rotation=0)
plt.legend(title='Usuário', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Exportar a análise de atividade por hora para um arquivo CSV
#activity_by_user_hour.to_csv('/Users/brenojaruzo/Downloads/activity_by_user_hour.csv')

In [ ]:
# Extraindo o dia da semana
df['weekday'] = df['date'].dt.day_name()

# Contagem de mensagens por dia da semana
weekday_counts = df['weekday'].value_counts()

# Ordenar os dias da semana na ordem correta
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_counts = weekday_counts.reindex(days_order)

# Plotar o gráfico
plt.figure(figsize=(10, 5))
weekday_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Distribuição de Mensagens por Dia da Semana', fontsize=16)
plt.xlabel('Dia da Semana', fontsize=12)
plt.ylabel('Número de Mensagens', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
